In [1]:
import numpy as np
import cv2
import math
np.seterr(over='ignore')

{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [2]:
def find_spatial_distance(x1, y1, x2, y2): #simple gaussian function, find spatial diff between given 2 number
    return np.sqrt((x1-x2)**2 + (y1-y2)**2)

In [3]:
def apply_gaussian(x, sigma): #simple gaussian function
    pi = math.pi
    e =  math.exp(1)
    return (1.0 / (2 * pi  * (sigma ** 2))) * e**(- (x ** 2) / (2 * sigma ** 2))

In [4]:
def find_intensity_diff(src, x1, y1, x2, y2):  #basically gives intensity difference between two pixel
    intensity_of_orginal_image = src[x1][y1]
    intensity_of_neighbour = src[x2][y2]
    return  intensity_of_orginal_image - intensity_of_neighbour

In [5]:
def find_normalization_term_wp(src, neighbour_x, neighbour_y, img_x, img_y, sigma_i, sigma_s): #gives Wp
    intensity_diff = find_intensity_diff(src, neighbour_x, neighbour_y, img_x, img_y)
    spatial_distance = find_spatial_distance(neighbour_x, neighbour_y, img_x, img_y)
        
    gi = apply_gaussian(intensity_diff, sigma_i) #apply gaussian for intensity_difference of neighbour and pixel
    gs = apply_gaussian(spatial_distance, sigma_s)#apply gaussian for spatial diff of neighbour and pixel
    
    normalization_term = gi * gs
    
    return normalization_term

In [6]:
def apply_filter(image, x, y, window_size, sigma_i, sigma_s):
    hl = window_size//2 #hl is 3 in our case, possible values are row +- 3 and col +- 3  
    filtered_pixel = 0
    Wp = 0
    
    for row in range(0,window_size):
        for col in range(0,window_size):
            neighbour_x = x - (hl - row)
            neighbour_y = y - (hl - col)
            
            #I USED EDGE OPTION FOR FILTERING AS REPLICATION OF BORDER PIXELS!!
            if neighbour_x < 0: #for negative x values, take borders' pixels
                neighbour_x = 0
            
            if neighbour_x >= len(image): #if we exceed the row count, again take borders' pixels
                neighbour_x = len(image) - 1
                
            if neighbour_y < 0: #for negative y values, take borders
                neighbour_y = 0
                
            if neighbour_y >= len(image[0]): #if we exceed the col count, again take borders' pixels
                neighbour_y = len(image[0]) - 1
            
            normalization_term_of_neighbour_pixel = find_normalization_term_wp(image, neighbour_x, neighbour_y, x, y, sigma_i, sigma_s)
            filtered_pixel += image[neighbour_x][neighbour_y] * normalization_term_of_neighbour_pixel
            Wp += normalization_term_of_neighbour_pixel #total Wp
    filtered_pixel /= Wp  #multiply summation with 1/Wp
    return filtered_pixel #returns filtered pixel intensity, 
                #and we will give that intensity to pixel at copy of original image

In [7]:
def bilateral_filter_main(image, window_size, sigma_i, sigma_s):
    filtered_image = np.zeros(image.shape) #copy of original image

    for row in range(0,len(image)):
        for col in range(0,len(image[0])): #pixel at (row,col) will be filtered
            filtered_image[row][col] = int(round(apply_filter(image, row, col, window_size, sigma_i, sigma_s)))
    
    return filtered_image

In [8]:
src = cv2.imread("in_img.jpg", 0)
cv2.imwrite("original_image_grayscale.png", src) #I wrote image as grayscale to compare with filtered ones

filtered_image_OpenCV = cv2.bilateralFilter(src, 7, 3.0, 14.0)
cv2.imwrite("OpenCV Filtered Image.png", filtered_image_OpenCV) #opencv filtered image

filtered_image_own = bilateral_filter_main(src, 7, 3.0, 14.0)
cv2.imwrite("My Filtered Image.png", filtered_image_own)  #my filtered image

True

#Our main goal of using bilateral filter is reducing the noise of an image while preserving edges.This smoothing filter has sigma values for intensity kernel and spatial kernel.After I read some documentations about bilateral filtering and try a lot of sigma values combinations,I determined best sigma values for intensity kernel(sigmaI) as 3 and spatial kernel(sigmaS) as 18.

The reason behing of this choice is that for example,when I keep sigmaS constant and increase sigmaI two by two,I saw that we are getting more blurred image,and close pixel intensities are mixing, so we cannot preserve edges for increased sigmaI values.
When I increased sigmaI, image started to become more blured after sigmaI = 6, and I decide to choose sigmaI less than 6.And when I increased sigmaI one by one starting at 2 and I saw that when sigmaI = 3,noise on image is decreased compared to simgaI = 2, and I decide to choose best sigmaI value as 3. sigmaI = 5 was making image more blurred
and we lost some edges and last options were 3 or 4. I tried 3 and 4 with different sigmaS combinations, actually 2 or 3 of that combinations made me a bit confused. 

Then I increased sigmaS two by two.When I reached sigmaS values more than 14 with sigmaI = 3,I found better results for smooting image while preserving edge goal.
I tried sigmaI = 3, sigmaS = 20 combination and this gave more blurred image compared to 3,18 combination and I decided to choose sigmaS less than 20 and more than 14.
I also compared 3,18 combination with 4,18 combination, 4,18 mixes some edges and I elected that combination.

Finally I decided to choose sigmaI = 3 and sigmaS = 18, and there were some combinations made me confused (3,14 and 4,14), but I think 3,18 was best.